In [1]:
import os, sys
sys.path.append('..') # Parent directory in path
from time import time, sleep
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import scipy.stats as stats
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import statsmodels.api as sm

pd.set_option("display.precision", 4)
#import pandas_market_calendars as mcal # NYSE Calendar

import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rc("font", **{"size": 14})
plt.rc("figure", **{"figsize": (16,10)})
# import matplotlib.pylab as pl
from matplotlib import cm

In [2]:
def hyperbola(x, a, b, c, d, e) -> float:
    """Tiltable hyperbola for scipy.optimize.curve_fit
    Asymptotes at 
    y=(-b/a+d)(x-c)+e
    y=( b/a+d)(x-c)+e"""
    return (b**2*(1+(x-c)**2/a**2))**0.5+d*(x-c)+e

def hyperbola_jac(x, a, b, c, d, e) -> np.ndarray:
    """Jacobian of the hyperbola with respect to each parameter"""
    ru = np.sqrt(b**2*(1+(x-c)**2/a**2))
    da = -(b**2 * (x-c)**2) / (a**3 * ru)
    db = np.sqrt(1+(x-c)**2/a**2)
    dc = -(b**2 * (x-c)) / (a**2 * ru) - d
    dd = x-c
    de = np.ones(len(x))
    return np.array([da, db, dc, dd, de]).T

def hyperbola_minimizer(a, b, c, d, e) -> float:
    """Returns the minimizer (x such that y is the minimum) of a hyperbola"""
    return ((a**2 * c * d**2 + d*(a**4 * b**2 - a**6 * d**2)**0.5 - b**2 * c)
            /(a**2 * d**2 - b**2))

def fit_hyp(df) -> np.ndarray:
    """Fits a hyperbola for a day's IV single curve"""
    curve = df.loc[:,["LOG_MONEYNESS_F","IV_MID"]].dropna()
    bounds = ((1e-4, 1e-4, 0, -2, -2), (1, 1, 1, 2, 2))
    try:
        popt, pcov = curve_fit(hyperbola, curve["LOG_MONEYNESS_F"], curve["IV_MID"]**2, 
                               jac=hyperbola_jac, bounds=bounds, maxfev=5000)
        return popt
    except (RuntimeError, ValueError):
        return np.array([np.nan] * 5)

In [3]:
all = pd.concat([pd.read_hdf(os.path.join("..", "data", "spx_iv_db_1.h5")), 
                 pd.read_hdf(os.path.join("..", "data", "spx_iv_db_2.h5"))])

TS_dt = all['TS'].unique()
writer = pd.ExcelWriter('./iv_parameter_TS.xlsx', engine = 'xlsxwriter')

In [4]:
for dt in TS_dt:
    print(dt)
    df = all[all['TS'] == dt].dropna().reset_index(drop=True)
    ATM_vols = df[df["RANK"]==0]
    df = pd.merge(df, ATM_vols[["TS", "TYPE", "IV_BID", "IV_ASK"]], left_on=["TS", "TYPE"], right_on=["TS", "TYPE"], 
                  suffixes=(None, "_ATM"), how="left")
    
    df["MONEYNESS"] = df["UNDERLYING_PRICE"] - df["STRIKE"]
    df["MONEYNESS_F"] = df["STRIKE"]/df["F_T"]
    df["LOG_MONEYNESS_F"] = np.log(df["MONEYNESS_F"])
    df["LOG_MONEYNESS_F_STANDARD_TIME"] = df["LOG_MONEYNESS_F"] / np.sqrt(df["CAL_DAYS"])

    # df["LOG_MONEYNESS_F_STANDARD_STD"] = df["LOG_MONEYNESS_F"] / (np.sqrt(df["CAL_DAYS"])*(df["IV_BID_ATM"]+df["IV_ASK_ATM"])/2)

    df["IV_BID_DIFF"] = df["IV_BID"] - df["IV_BID_ATM"]
    df["IV_ASK_DIFF"] = df["IV_ASK"] - df["IV_ASK_ATM"]

    df["IV_BID_RATIO"] = df["IV_BID"]/df["IV_BID_ATM"]
    df["IV_ASK_RATIO"] = df["IV_ASK"]/df["IV_ASK_ATM"]
    
    otm_mask = (((df["TYPE"]=='C') & (df["STRIKE"]>df["UNDERLYING_PRICE"])) 
            | ((df["TYPE"]=='P') & (df["STRIKE"]<=df["UNDERLYING_PRICE"])))

    otm  = df[otm_mask].copy().reset_index(drop=True)
    to_fit = otm

    fits = to_fit.groupby("EXP").apply(fit_hyp)
    fits = np.array([v for v in fits.values])

    fit_df = pd.DataFrame(fits, index=to_fit["EXP"].unique(), columns=['a', 'b', 'c', 'd', 'e'])
    fit_df.to_excel(writer, sheet_name=pd.to_datetime(str(dt)).strftime('%Y-%m-%d'))

2017-07-03T00:00:00.000000000
2017-07-05T00:00:00.000000000
2017-07-06T00:00:00.000000000
2017-07-07T00:00:00.000000000
2017-07-10T00:00:00.000000000
2017-07-11T00:00:00.000000000
2017-07-12T00:00:00.000000000
2017-07-13T00:00:00.000000000
2017-07-14T00:00:00.000000000
2017-07-17T00:00:00.000000000
2017-07-18T00:00:00.000000000
2017-07-19T00:00:00.000000000
2017-07-20T00:00:00.000000000
2017-07-21T00:00:00.000000000
2017-07-24T00:00:00.000000000
2017-07-25T00:00:00.000000000
2017-07-26T00:00:00.000000000
2017-07-27T00:00:00.000000000
2017-07-28T00:00:00.000000000
2017-07-31T00:00:00.000000000
2017-08-01T00:00:00.000000000
2017-08-02T00:00:00.000000000
2017-08-03T00:00:00.000000000
2017-08-04T00:00:00.000000000
2017-08-07T00:00:00.000000000
2017-08-08T00:00:00.000000000
2017-08-09T00:00:00.000000000
2017-08-10T00:00:00.000000000
2017-08-11T00:00:00.000000000
2017-08-14T00:00:00.000000000
2017-08-15T00:00:00.000000000
2017-08-16T00:00:00.000000000
2017-08-17T00:00:00.000000000
2017-08-18

2018-08-03T00:00:00.000000000
2018-08-06T00:00:00.000000000
2018-08-07T00:00:00.000000000
2018-08-08T00:00:00.000000000
2018-08-09T00:00:00.000000000
2018-08-10T00:00:00.000000000
2018-08-13T00:00:00.000000000
2018-08-14T00:00:00.000000000
2018-08-15T00:00:00.000000000
2018-08-16T00:00:00.000000000
2018-08-17T00:00:00.000000000
2018-08-20T00:00:00.000000000
2018-08-21T00:00:00.000000000
2018-08-22T00:00:00.000000000
2018-08-23T00:00:00.000000000
2018-08-24T00:00:00.000000000
2018-08-27T00:00:00.000000000
2018-08-28T00:00:00.000000000
2018-08-29T00:00:00.000000000
2018-08-30T00:00:00.000000000
2018-08-31T00:00:00.000000000
2018-09-04T00:00:00.000000000
2018-09-05T00:00:00.000000000
2018-09-06T00:00:00.000000000
2018-09-07T00:00:00.000000000
2018-09-10T00:00:00.000000000
2018-09-11T00:00:00.000000000
2018-09-12T00:00:00.000000000
2018-09-13T00:00:00.000000000
2018-09-14T00:00:00.000000000
2018-09-17T00:00:00.000000000
2018-09-18T00:00:00.000000000
2018-09-19T00:00:00.000000000
2018-09-20

2019-09-06T00:00:00.000000000
2019-09-09T00:00:00.000000000
2019-09-10T00:00:00.000000000
2019-09-11T00:00:00.000000000
2019-09-12T00:00:00.000000000
2019-09-13T00:00:00.000000000
2019-09-16T00:00:00.000000000
2019-09-17T00:00:00.000000000
2019-09-18T00:00:00.000000000
2019-09-19T00:00:00.000000000
2019-09-20T00:00:00.000000000
2019-09-23T00:00:00.000000000
2019-09-24T00:00:00.000000000
2019-09-25T00:00:00.000000000
2019-09-26T00:00:00.000000000
2019-09-27T00:00:00.000000000
2019-09-30T00:00:00.000000000
2019-10-01T00:00:00.000000000
2019-10-02T00:00:00.000000000
2019-10-03T00:00:00.000000000
2019-10-04T00:00:00.000000000
2019-10-07T00:00:00.000000000
2019-10-08T00:00:00.000000000
2019-10-09T00:00:00.000000000
2019-10-10T00:00:00.000000000
2019-10-11T00:00:00.000000000
2019-10-14T00:00:00.000000000
2019-10-15T00:00:00.000000000
2019-10-16T00:00:00.000000000
2019-10-17T00:00:00.000000000
2019-10-18T00:00:00.000000000
2019-10-21T00:00:00.000000000
2019-10-22T00:00:00.000000000
2019-10-23

2020-10-08T00:00:00.000000000
2020-10-09T00:00:00.000000000
2020-10-12T00:00:00.000000000
2020-10-13T00:00:00.000000000
2020-10-14T00:00:00.000000000
2020-10-15T00:00:00.000000000
2020-10-16T00:00:00.000000000
2020-10-19T00:00:00.000000000
2020-10-20T00:00:00.000000000
2020-10-21T00:00:00.000000000
2020-10-22T00:00:00.000000000
2020-10-23T00:00:00.000000000
2020-10-26T00:00:00.000000000
2020-10-27T00:00:00.000000000
2020-10-28T00:00:00.000000000
2020-10-29T00:00:00.000000000
2020-10-30T00:00:00.000000000
2020-11-02T00:00:00.000000000
2020-11-03T00:00:00.000000000
2020-11-04T00:00:00.000000000
2020-11-05T00:00:00.000000000
2020-11-06T00:00:00.000000000
2020-11-09T00:00:00.000000000
2020-11-10T00:00:00.000000000
2020-11-11T00:00:00.000000000
2020-11-12T00:00:00.000000000
2020-11-13T00:00:00.000000000
2020-11-16T00:00:00.000000000
2020-11-17T00:00:00.000000000
2020-11-18T00:00:00.000000000
2020-11-19T00:00:00.000000000
2020-11-20T00:00:00.000000000
2020-11-23T00:00:00.000000000
2020-11-24

2021-11-09T00:00:00.000000000
2021-11-10T00:00:00.000000000
2021-11-11T00:00:00.000000000
2021-11-12T00:00:00.000000000
2021-11-15T00:00:00.000000000
2021-11-16T00:00:00.000000000
2021-11-17T00:00:00.000000000
2021-11-18T00:00:00.000000000
2021-11-19T00:00:00.000000000
2021-11-22T00:00:00.000000000
2021-11-23T00:00:00.000000000
2021-11-24T00:00:00.000000000
2021-11-26T00:00:00.000000000
2021-11-29T00:00:00.000000000
2021-11-30T00:00:00.000000000
2021-12-01T00:00:00.000000000
2021-12-02T00:00:00.000000000
2021-12-03T00:00:00.000000000
2021-12-06T00:00:00.000000000
2021-12-07T00:00:00.000000000
2021-12-08T00:00:00.000000000
2021-12-09T00:00:00.000000000
2021-12-10T00:00:00.000000000
2021-12-13T00:00:00.000000000
2021-12-14T00:00:00.000000000
2021-12-15T00:00:00.000000000
2021-12-16T00:00:00.000000000
2021-12-17T00:00:00.000000000
2021-12-20T00:00:00.000000000
2021-12-21T00:00:00.000000000
2021-12-22T00:00:00.000000000
2021-12-23T00:00:00.000000000
2021-12-27T00:00:00.000000000
2021-12-28

In [5]:
writer.close()